### Skip to the last cell to run the best player found by SA_CEM

In [1]:
import random
import heapq
from numpy.linalg import cholesky
import numpy as np
from collections import deque
from tetris_utils import *
from tetris import *
from copy import deepcopy
import pygame,sys
import os
import matplotlib.pyplot as plt
from tetris import TetrisApp
App=TetrisApp()
from tetris_reinforcement_learner import TetrisReinforcementLearner
RL_learner=TetrisReinforcementLearner(App)


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:

done = False
batch_size = 40


state_size = 270
action_size = 30
# agent.load("./save/cartpole-dqn.h5")
App.init_game()
def pixalize_stone(stone):
    container=np.zeros((2,10))
    if len(stone)==1:
        container[0][...,3:3+len(stone[0])]=stone[0]
    elif stone[0][0]==7:
        container[0][...,4:4+len(stone[0])]=stone[0]
        container[1][...,4:4+len(stone[0])]=stone[1]
    else:
        container[0][...,3:3+len(stone[0])]=stone[0]
        container[1][...,3:3+len(stone[0])]=stone[1]
    return container
    
def preprocess_data(state):
    
    board=np.array(state['board'])
    stone=state['stone']
    next_stone=state['next_stone']
    pixal_stone=pixalize_stone(stone)
    pixal_next_stone=pixalize_stone(next_stone)

    final_board=np.append(pixal_next_stone,pixal_stone,axis=0)
    final_board=np.append(final_board,board,axis=0)
    final_board=np.where(final_board>0,1,0)
#     print(final_board)
    final_board=final_board.flatten()
    final_board=final_board.reshape(1,270)
    return final_board


def update_new(reinforcement_learner, old_state,new_state):## Taking holes into account
    change_in_pile_height = reinforcement_learner.get_pile_height(new_state) - reinforcement_learner.get_pile_height(old_state)
    reward = -1 * change_in_pile_height
    return reward# container=np.zeros((len(stone),10))
# for i in range(len(stone)):
#     container[i][...,3:3+len(stone[0])]=stone[i]
# board=board.flatten()
# board=board[:,np.newaxis]
# board = np.where(board>0,1,0)
# stone_idx=find_stone_idx(stone)
# next_stone_idx=find_stone_idx(state['next_stone'])

In [3]:
def get_feature(state):
    height=RL_learner.get_pile_height(state)
    holes=RL_learner.get_holes(state)
    contours=RL_learner.get_contours(state)
    row_holes=RL_learner.get_row_holes(state)
    features=np.array([[height],[holes],[contours],[row_holes]])
    return features
def get_current_state_CEM_value(state,action_sequence):
    successor_state=RL_learner.get_successor_state(state,action_sequence)
    features=get_feature(successor_state)
    for feature, value in features.items():
        CEM_val+=RL_learner.CEM_weights[feature]*value
        
def get_following_state_CEM_value(state,action_sequence,cur_theta):
    
    successor_state=RL_learner.get_successor_state(state,action_sequence)
    next_state_reward=RL_learner.get_next_action_reward(state,action_sequence)
#     print('next_state_reward is',next_state_reward)
    done=successor_state['gameover']
    if not done:
        
        features=get_feature(successor_state)
        features=np.append(features,np.array([[next_state_reward]]),axis=0)
    return np.squeeze(cur_theta.T.dot(features))

In [4]:
def get_sample_theta(mu,sigma,num_sample):
    R = cholesky(sigma)
    s = np.dot(np.random.randn(num_sample, 5), R) + mu.T
    return s

statistic=[]
statistic1=[]
def mysoftmaxchoice(x):
    expx=np.exp(x)
    expx=expx/max(expx)
    smx=expx/sum(expx)
    return np.random.choice(len(x), size=None, replace=True, p=np.array(smx))

In [5]:
#top_5_theta = deque(maxlen=5)
mu0=np.zeros((5,1)) #initialize mu
sigma0=np.eye(5,5) #initialize theta
num_sample=50
s=get_sample_theta(mu0,100*sigma0,num_sample)
num_repeat=1

theta_grade=[] # store the grade each theta earned
for theta_idx in range(num_sample):
    R_temp=0
    print(theta_idx,end="\r")
    cur_theta=np.array(s[theta_idx])
            #cur_theta=cur_theta[:,np.newaxis]
    for i in range(num_repeat):
        App.init_game()
        if not App.pygame_initted: App.init_pygame()
        App.gameover=False
        App.paused=False
        App.start_game()
        done=False
        while not done:
            App.display_board()
            state=RL_learner.capture_state_attributes(App)
            dont_burn_my_cpu=pygame.time.Clock()  

            ## calculate the next state value, which is store in the list: next_val
            next_val=[] # store the next_state_value
            legal_action_sequences=RL_learner.get_legal_action_sequences(state)
            for actions in legal_action_sequences:
                next_val.append(get_following_state_CEM_value(state,actions,cur_theta))
            next_action=legal_action_sequences[np.argmax(next_val)]
            App.play_action_sequence(next_action)
            new_state=RL_learner.capture_state_attributes(App)
            done=new_state['gameover']
        
        if done:
            R_temp=R_temp+App.lines
                
    theta_grade.append(R_temp/num_repeat)
#top_5_theta_idx=list(map(theta_grade.index, heapq.nlargest(5, theta_grade)))
#for idx in top_5_theta_idx:
#    top_5_theta.append(s[idx])

KeyboardInterrupt: 

In [7]:
top_50_theta=s
top_50_grade=theta_grade
#top_50_theta_idx=list(map(theta_grade.tolist().index, heapq.nlargest(50, theta_grade)))
#for idx in top_50_theta_idx:
#    top_50_theta.append(s[idx])
#    top_50_grade.append(theta_grade[idx])

In [27]:
#statistic1=np.loadtxt("statistics.txt")
#statistic1=statistic1.tolist()
#top_50_grade=np.loadtxt("theta_grade.txt")
#top_50_theta=np.loadtxt("theta.txt")

#num_sample=100
T0=.15
num_repeat=1
for i in range(12):
    np_top_50_theta=np.array(top_50_theta).T
    mu=np.mean(np_top_50_theta,axis=1)
    sigma=np.cov(np_top_50_theta)+np.eye(5)*1e-16#+max(5-(i+1)/10,0)*np.eye(3)
    
    s_cem=get_sample_theta(mu,sigma,50)
    
    T=T0-i/100
    ds=20*T*get_sample_theta(mu0,sigma0,50)
    
    s_sa=top_50_theta+ds
    #s=np.vstack((np_top_50_theta+ds,s50))
    #theta_grade=[] # store the grade each theta earned
    #candidate=np.zeros((5,0))
    #candidate_grade=[]
    #top_50_theta=[]
    #top_50_grade=[]
    for theta_idx in range(50):
        R_temp=0
        print('iter: ',i,', idx: ',theta_idx,end="\r")
        for j in range(num_repeat):
            
            App.init_game()
            if not App.pygame_initted: App.init_pygame()
            App.gameover=False
            App.paused=False
            App.start_game()
            done=False
            cur_theta=s_sa[theta_idx]
            #cur_theta=cur_theta[:,np.newaxis]
            while not done:
                App.display_board()
                state=RL_learner.capture_state_attributes(App)
                dont_burn_my_cpu=pygame.time.Clock()  

                ## calculate the next state value, which is store in the list: next_val
                next_val=[] # store the next_state_value
                legal_action_sequences=RL_learner.get_legal_action_sequences(state)
                for actions in legal_action_sequences:
                    next_val.append(get_following_state_CEM_value(state,actions,cur_theta))
                next_action=legal_action_sequences[np.argmax(next_val)]
                App.play_action_sequence(next_action)
                new_state=RL_learner.capture_state_attributes(App)
                done=new_state['gameover']
            
            if done:
                R_temp=R_temp+App.lines
        
        R_temp=R_temp/num_repeat
        deltaR=(R_temp+1e-16)/(top_50_grade[theta_idx]+1e-16)-1
        if deltaR>0:
            top_50_theta[theta_idx]=cur_theta
            top_50_grade[theta_idx]=R_temp
        else:
            pa=np.exp(deltaR/T)
            if np.random.choice([True, False], size=None, replace=True, p=[pa, 1-pa]):
                top_50_theta[theta_idx]=cur_theta
                top_50_grade[theta_idx]=R_temp
            #else:
            #    candidate=np.vstack((candidate,cur_theta))
            #    candidate_grade.append(R_temp)
    
    #cem_theta=[]
    cem_grade=np.zeros(50)
    for theta_idx in range(50):
        R_temp=0
        print('iter: ',i,', idx: ',theta_idx+50,end="\r")
        for j in range(num_repeat):
            
            App.init_game()
            if not App.pygame_initted: App.init_pygame()
            App.gameover=False
            App.paused=False
            App.start_game()
            done=False
            cur_theta=s_cem[theta_idx]
            #cur_theta=cur_theta[:,np.newaxis]
            while not done:
                App.display_board()
                state=RL_learner.capture_state_attributes(App)
                dont_burn_my_cpu=pygame.time.Clock()  

                ## calculate the next state value, which is store in the list: next_val
                next_val=[] # store the next_state_value
                legal_action_sequences=RL_learner.get_legal_action_sequences(state)
                for actions in legal_action_sequences:
                    next_val.append(get_following_state_CEM_value(state,actions,cur_theta))
                next_action=legal_action_sequences[np.argmax(next_val)]
                App.play_action_sequence(next_action)
                new_state=RL_learner.capture_state_attributes(App)
                done=new_state['gameover']
            
            if done:
                R_temp=R_temp+App.lines
        cem_grade[theta_idx]=R_temp/num_repeat
    
    s=np.vstack((top_50_theta,s_cem))

    theta_grade_array=np.hstack((top_50_grade,cem_grade))
    theta_grade=theta_grade_array.tolist()
            
    top_50_theta_idx=list(map(theta_grade.index, heapq.nlargest(50, theta_grade)))
    
    top_50_theta=s[top_50_theta_idx]
    top_50_grade=theta_grade_array[top_50_theta_idx]
      
    statistic1.append([np.max(theta_grade),np.min(theta_grade),np.mean(theta_grade)])
    #print(i)
    print(statistic1)
    statistic_np=np.array(statistic1)
    np.savetxt("statistics.txt",statistic_np)
    theta_grade_np=np.array(theta_grade)
    np.savetxt("theta_grade.txt",top_50_grade)
    theta_np=np.array(top_50_theta)
    np.savetxt("theta.txt",theta_np)

[[85.0, 0.0, 6.98], [288.0, 0.0, 20.0]]


KeyboardInterrupt: 

In [ ]:
#Best test
best_theta=[4.231203317932727970e+00, -2.039093680914587736e+01, -4.425357456410593748e+00, 8.171306159156509707e+00, 8.939371296855847859e+00
]
num_repeat=10
R_temp=0
T0=.01
for j in range(num_repeat):
    App.init_game()
    if not App.pygame_initted: App.init_pygame()
    App.gameover=False
    App.paused=False
    App.start_game()
    done=False
    cur_theta=np.array(best_theta)
    #cur_theta=cur_theta[:,np.newaxis]
    while not done:
        App.display_board()
        state=RL_learner.capture_state_attributes(App)
        dont_burn_my_cpu=pygame.time.Clock()  

        ## calculate the next state value, which is store in the list: next_val
        next_val=[] # store the next_state_value
        legal_action_sequences=RL_learner.get_legal_action_sequences(state)
        for actions in legal_action_sequences:
            next_val.append(get_following_state_CEM_value(state,actions,cur_theta))
        next_action=legal_action_sequences[np.argmax(next_val)]
        App.play_action_sequence(next_action)
        new_state=RL_learner.capture_state_attributes(App)
        done=new_state['gameover']

    if done:
        R_temp=R_temp+App.lines
R_ave=R_temp/num_repeat
print(R_ave)